In [1]:
!pip install netCDF4
from google.colab import drive
drive.mount('/content/gdrive')
import netCDF4 as nc
import numpy as np
import seaborn as sns
import pandas as pd
from shapely.geometry import Polygon
import shapely
import matplotlib.pyplot as plt
import os
from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
dataset_1 = '/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/agr_data/Wind-Speed-10m-Mean/Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160101-20160131_final-v1.0.nc'
dataset1 = nc.Dataset(dataset_1)

In [11]:
lats = dataset1.variables['lat'][:]
lons = dataset1.variables['lon'][:]
lat_bnds, lon_bnds = [-11.183, 3.622], [28.417, 41.194]

lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
lat_inds = np.asarray(lat_inds)

lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))
lon_inds = np.asarray(lon_inds)

lat_list = np.ma.getdata(dataset1.variables['lat'][lat_inds[0,0]:lat_inds[0,-1]])
lon_list = np.ma.getdata(dataset1.variables['lon'][lon_inds[0,0]:lon_inds[0,-1]])

lat_interp = np.arange(start = np.round(lat_list[0], 2), stop = np.round(lat_list[-1], 2), step = -0.05)
lon_interp = np.arange(start = lon_list[0], stop = lon_list[-1], step = 0.05)

In [12]:
file_names = ['Wind-Speed-10m-Mean', 'Vapour-Pressure-Mean', 'Solar-Radiation-Flux', 'Cloud-Cover-Mean', 'Temperature-Air-2m-Mean-24h', 'Relative-Humidity-2m-12h']

for name in file_names:
  folder_string = '/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/agr_data/' + name
  files = os.listdir(folder_string)
  sorted_files = sorted(files)
  keys = list(nc.Dataset(folder_string + '/' + sorted_files[0]).variables.keys())[3]
  print(name, 'NAME')
  sorted_data = []
  for files in sorted_files:
    #data_file = nc.Dataset(folder_string + '/' + files).variables[keys][:, lat_inds[0, 0]:lat_inds[0, -1], lon_inds[0, 0] : lon_inds[0, -1]].filled(np.nan)
    ds = xr.open_dataset(folder_string + '/' + files).sel(lat = lat_list, lon = lon_list, method = 'nearest')
    ds_clip_int = ds.interp(lat = lat_interp, lon = lon_interp, method = 'nearest')
    sorted_data.append(ds_clip_int)
    print(files)
  merged = xr.concat(sorted_data, dim = 'time')
  merged.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/populate_grid/ERA5/agr_data/' + 'merged_agg' + name + '.nc')


Wind-Speed-10m-Mean NAME
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160101-20160131_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160201-20160229_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160301-20160331_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160401-20160430_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160501-20160531_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160601-20160630_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160701-20160731_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160801-20160831_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20160901-20160930_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20161001-20161031_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20161101-20161130_final-v1.0.nc
Wind-Speed-10m-Mean_C3S-glob-agric_AgERA5_daily_20161201-20161231_final-v1.0.nc
Wind-Speed-10m-

In [8]:
xr.concat(sorted_data, dim = 'time')

<xarray.Dataset>
Dimensions:              (lat: 316, lon: 276, time: 335)
Coordinates:
  * lat                  (lat) float64 3.9 3.85 3.8 3.75 ... -11.75 -11.8 -11.85
  * lon                  (lon) float64 28.1 28.15 28.2 ... 41.75 41.8 41.85
  * time                 (time) datetime64[ns] 2016-01-01 ... 2016-11-30
Data variables:
    Wind_Speed_10m_Mean  (time, lat, lon) float32 1.7661473 1.766192 ... nan nan
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    history:      Fri Mar 01 10:55:48 2019: cdo -z zip_4 -f nc4 -O mergetime ...
    Conventions:  CF-1.7
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...

In [7]:
sorted_data

[<xarray.Dataset>
 Dimensions:              (lat: 316, lon: 276, time: 31)
 Coordinates:
   * time                 (time) datetime64[ns] 2016-01-01 ... 2016-01-31
   * lat                  (lat) float64 3.9 3.85 3.8 3.75 ... -11.75 -11.8 -11.85
   * lon                  (lon) float64 28.1 28.15 28.2 ... 41.75 41.8 41.85
 Data variables:
     Wind_Speed_10m_Mean  (time, lat, lon) float32 1.7661473 1.766192 ... nan nan
 Attributes:
     CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
     history:      Fri Mar 01 10:55:48 2019: cdo -z zip_4 -f nc4 -O mergetime ...
     Conventions:  CF-1.7
     CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...,
 <xarray.Dataset>
 Dimensions:              (lat: 316, lon: 276, time: 29)
 Coordinates:
   * time                 (time) datetime64[ns] 2016-02-01 ... 2016-02-29
   * lat                  (lat) float64 3.9 3.85 3.8 3.75 ... -11.75 -11.8 -11.85
   * lon                  (lon) float64 28.1 28.